In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn.metrics import mean_absolute_error
from collections import defaultdict

# SpecTE

In [3]:
# 结果融合生成一个结果  然后输出到r'F:/My_trial/paper/if_pretrain/'
with open(r'G:\Star\2_SpecTE\model_log\blending\5_50\Raw_Results_5_50.pkl', 'rb') as file:
    result_low = pickle.load(file)
with open(r'G:\Star\2_SpecTE\model_log\blending\50_999\Raw_Results_50_999.pkl', 'rb') as file:
    result_high = pickle.load(file)

para_dict = {
    'te':['Teff[K]'],
    'Lo':['Logg'],
    'RV':['RV'],
    'Fe':['FeH'],
    'Mg':['MgH'],
    'Si':['SiH'],
    'SH':['SH'],
    'KH':['KH'],
    'Ca':['CaH'],
    'Ti':['TiH'],
    'Cr':['CrH'],
    'Ni':['NiH'],
    'CH':['CH'],
    'NH':['NH'],
    'OH':['OH'],
    'Al':['AlH'],
    'Mn':['MnH'],
    'Na':['NaH'],
    'VH':['VH'],   
}
    
all_result = {key: [np.concatenate((result_low[key][0], result_high[key][0])),
                    np.concatenate((result_low[key][1], result_high[key][1]))] for key in para_dict.keys()}
all_result['snrg'] = [np.concatenate((result_low['snrg'][0], result_high['snrg'][0]), axis=0),]
# all_result=result_low

# 统计 MAE等  
df = pd.DataFrame()

for key, value in para_dict.items():  

    diff_std = (all_result[key][0] - all_result[key][1]).std(axis=0)
    diff_mean = (all_result[key][0] - all_result[key][1]).mean(axis=0)

    mae = mean_absolute_error(all_result[key][0], all_result[key][1], multioutput='raw_values')
    mae = mae[0] if len(mae) == 1 else np.mean(mae)

    # print(key," MAE:",mae)
    # print(key," σ:",diff_std)
    # print(key," μ:",diff_mean)
    df[str(value)] = pd.Series({'μ': diff_mean, 'σ': diff_std, 'MAE': mae})    

df = df.T
print(df)
# 直接保存统计结果
df.to_csv('Statistical_Results.csv', index=True)       

                        μ          σ        MAE
['Teff[K]']  1.979360e-01  90.231090  45.095743
['Logg']    -3.205570e-07   0.159631   0.080246
['RV']       2.269561e-02   9.884436   3.870888
['FeH']      5.515922e-04   0.064228   0.036682
['MgH']      5.609706e-04   0.063303   0.038901
['SiH']      3.799867e-04   0.065311   0.040091
['SH']       1.527555e-03   0.107936   0.070802
['KH']      -1.345930e-03   0.133444   0.076607
['CaH']      8.824571e-04   0.072601   0.044199
['TiH']      1.461952e-03   0.149820   0.092090
['CrH']      7.226204e-04   0.205611   0.117701
['NiH']      3.682520e-04   0.073280   0.043492
['CH']       2.330469e-04   0.080890   0.048257
['NH']       9.274504e-04   0.171172   0.098347
['OH']       3.343580e-04   0.096442   0.062337
['AlH']     -4.944452e-04   0.087806   0.053271
['MnH']      1.202636e-03   0.094198   0.054609
['NaH']      1.174926e-03   0.334407   0.209117
['VH']      -3.518991e-04   0.233836   0.152580


In [4]:
# 保存原始结果
# 转换为普通字典
simple_dict = {k: v for k, v in all_result.items()}
with open('./SpecTE_Raw_Results.pkl', 'wb') as file:
    pickle.dump(simple_dict, file)